In [ ]:
import requests
import os
import shutil
import zipfile
from os.path import join, getsize
import pandas as pd
import numpy as np
from threading import Thread
from concurrent.futures import ThreadPoolExecutor
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cd /content/drive/MyDrive/3001_Proj/preprocessed/

In [ ]:
def create_hour_csv(path):
    
    raw_dt= pd.read_csv(path)
    station_cluster = pd.read_csv('/content/drive/MyDrive/3001_Proj/preprocessed/intersection_station_cluster.csv') 
    cluster_dict = dict(zip(station_cluster['station_id'], 
                            station_cluster['cluster']))
    
    month_dt= raw_dt[(raw_dt['start_station_id'].isin(station_cluster['station_id']) )& (raw_dt['end_station_id'].isin(station_cluster['station_id']))]
    month_dt['start_station_id']= month_dt['start_station_id'].map(cluster_dict)
    month_dt['end_station_id']= month_dt['end_station_id'].map(cluster_dict)
    date_ls= sorted(list(month_dt['started_date'].unique())) 
    print(date_ls)
    ##############################################################################
    
    month_matrix=[]
    hour_ls=[]
    full_date_ls=[]
    
    
    for date in date_ls:
        date_dt= month_dt[month_dt['started_date']==date]
        for hour in np.arange(start=0, stop= 24, step=2):
            hour_dt= date_dt[(date_dt['started_hour']<=hour+1) 
                             & (date_dt['started_hour']>= hour) 
                             &(date_dt['ended_hour']<=(hour+2))]
            count_hour_dt= hour_dt.groupby(by= ['start_station_id', 'end_station_id'], as_index=False)['ride_id'].count()
            for i in range(6):
                if (i not in set(count_hour_dt['start_station_id'])) or (i not in set(count_hour_dt['end_station_id'])):
                    count_hour_dt = count_hour_dt.append({'start_station_id':i, 'end_station_id':i, 'ride_id':-1},ignore_index=True)
            hour_pivot_dt= count_hour_dt.pivot(index='start_station_id', columns='end_station_id')['ride_id'].fillna(0)

            hour_pivot_dt[hour_pivot_dt<0] = 0
            hour_ls.append(hour)
            full_date_ls.append(date)
            month_matrix.append(hour_pivot_dt)
#         hour_pivot_dt.to_csv('{d}-{h}.csv'.format(d = date, h=hour))
    return month_matrix, hour_ls, full_date_ls

In [ ]:
# import training 
paths=['/content/drive/MyDrive/3001_Proj/preprocessed/202102-citibike-tripdata_pre_analyzed.csv',
'/content/drive/MyDrive/3001_Proj/preprocessed/202103-citibike-tripdata_pre_analyzed.csv',
'/content/drive/MyDrive/3001_Proj/preprocessed/202104-citibike-tripdata_pre_analyzed.csv',
'/content/drive/MyDrive/3001_Proj/preprocessed/202105-citibike-tripdata_pre_analyzed.csv',
'/content/drive/MyDrive/3001_Proj/preprocessed/202106-citibike-tripdata_pre_analyzed.csv',
'/content/drive/MyDrive/3001_Proj/preprocessed/202107-citibike-tripdata_pre_analyzed.csv',
'/content/drive/MyDrive/3001_Proj/preprocessed/202108-citibike-tripdata_pre_analyzed.csv',
'/content/drive/MyDrive/3001_Proj/preprocessed/202109-citibike-tripdata_pre_analyzed.csv',
'/content/drive/MyDrive/3001_Proj/preprocessed/202110-citibike-tripdata_pre_analyzed.csv',
'/content/drive/MyDrive/3001_Proj/preprocessed/202111-citibike-tripdata_pre_analyzed.csv']


In [ ]:
from sklearn.ensemble import RandomForestRegressor as RFR

In [ ]:
reg = RFR(n_estimators=100 
          ,max_depth= 8 
          ,random_state=1024
          ,warm_start=True
          ,verbose=False
          ,n_jobs=-1  
         )
 

In [ ]:
!cd /content/drive/MyDrive/3001_Proj/preprocessed/

In [ ]:
start = time.time()

for p in paths:
    month_matrix, hour_ls, full_date_ls= create_hour_csv(p)
    full_dt= pd.DataFrame()

    for i in range(len(month_matrix)-1):
      temp= pd.DataFrame((month_matrix[i+1]- month_matrix[i].T).unstack().reset_index())
      temp['date']= full_date_ls[i]
      temp['previous_hour']= hour_ls[i]

      full_dt= pd.concat([full_dt, temp])

    full_dt.columns= ['station1', 'station2', 'trans_num', 'date', 'previous_hour']
    full_dt['date']= pd.to_datetime(full_dt['date']) 
    full_dt['day']= full_dt['date'].dt.day 
    X_train= full_dt.drop(columns=['trans_num', 'date'])
    y_train= full_dt['trans_num']

    newtree = 60
    reg.n_estimators += newtree
    reg = reg.fit(X_train,y_train)
     
end = time.time()
print(end-start)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


['2021-02-01', '2021-02-03', '2021-02-04', '2021-02-05', '2021-02-06', '2021-02-07', '2021-02-08', '2021-02-09', '2021-02-10', '2021-02-11', '2021-02-12', '2021-02-13', '2021-02-14', '2021-02-15', '2021-02-16', '2021-02-17', '2021-02-18', '2021-02-19', '2021-02-20', '2021-02-21', '2021-02-22', '2021-02-23', '2021-02-24', '2021-02-25', '2021-02-26', '2021-02-27', '2021-02-28']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


['2021-03-01', '2021-03-02', '2021-03-03', '2021-03-04', '2021-03-05', '2021-03-06', '2021-03-07', '2021-03-08', '2021-03-09', '2021-03-10', '2021-03-11', '2021-03-12', '2021-03-13', '2021-03-14', '2021-03-15', '2021-03-16', '2021-03-17', '2021-03-18', '2021-03-19', '2021-03-20', '2021-03-21', '2021-03-22', '2021-03-23', '2021-03-24', '2021-03-25', '2021-03-26', '2021-03-27', '2021-03-28', '2021-03-29', '2021-03-30', '2021-03-31']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


...

['2021-09-01', '2021-09-02', '2021-09-03', '2021-09-04', '2021-09-05', '2021-09-06', '2021-09-07', '2021-09-08', '2021-09-09', '2021-09-10', '2021-09-11', '2021-09-12', '2021-09-13', '2021-09-14', '2021-09-15', '2021-09-16', '2021-09-17', '2021-09-18', '2021-09-19', '2021-09-20', '2021-09-21', '2021-09-22', '2021-09-23', '2021-09-24', '2021-09-25', '2021-09-26', '2021-09-27', '2021-09-28', '2021-09-29', '2021-09-30']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


['2021-10-01', '2021-10-02', '2021-10-03', '2021-10-04', '2021-10-05', '2021-10-06', '2021-10-07', '2021-10-08', '2021-10-09', '2021-10-10', '2021-10-11', '2021-10-12', '2021-10-13', '2021-10-14', '2021-10-15', '2021-10-16', '2021-10-17', '2021-10-18', '2021-10-19', '2021-10-20', '2021-10-21', '2021-10-22', '2021-10-23', '2021-10-24', '2021-10-25', '2021-10-26', '2021-10-27', '2021-10-28', '2021-10-29', '2021-10-30', '2021-10-31']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


['2021-11-01', '2021-11-02', '2021-11-03', '2021-11-04', '2021-11-05', '2021-11-06', '2021-11-07', '2021-11-08', '2021-11-09', '2021-11-10', '2021-11-11', '2021-11-12', '2021-11-13', '2021-11-14', '2021-11-15', '2021-11-16', '2021-11-17', '2021-11-18', '2021-11-19', '2021-11-20', '2021-11-21', '2021-11-22', '2021-11-23', '2021-11-24', '2021-11-25', '2021-11-26', '2021-11-27', '2021-11-28', '2021-11-29', '2021-11-30']
180.20040011405945


In [ ]:
for p in ['/content/drive/MyDrive/3001_Proj/preprocessed/202112-citibike-tripdata_pre_analyzed.csv']:
    month_matrix, hour_ls, full_date_ls= create_hour_csv(p)
    full_dt= pd.DataFrame()

    for i in range(len(month_matrix)-1):
      temp= pd.DataFrame((month_matrix[i+1]- month_matrix[i].T).unstack().reset_index())
      temp['date']= full_date_ls[i]
      temp['previous_hour']= hour_ls[i]

      full_dt= pd.concat([full_dt, temp])

    full_dt.columns= ['station1', 'station2', 'trans_num', 'date', 'previous_hour']
    full_dt['date']= pd.to_datetime(full_dt['date']) 
    full_dt['day']= full_dt['date'].dt.day 
    X_test= full_dt.drop(columns=['trans_num', 'date'])
    y_test= full_dt['trans_num']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


['2021-12-01', '2021-12-02', '2021-12-03', '2021-12-04', '2021-12-05', '2021-12-06', '2021-12-07', '2021-12-08', '2021-12-09', '2021-12-10', '2021-12-11', '2021-12-12', '2021-12-13', '2021-12-14', '2021-12-15', '2021-12-16', '2021-12-17', '2021-12-18', '2021-12-19', '2021-12-20', '2021-12-21', '2021-12-22', '2021-12-23', '2021-12-24', '2021-12-25', '2021-12-26', '2021-12-27', '2021-12-28', '2021-12-29', '2021-12-30', '2021-12-31']


In [ ]:
from sklearn.metrics import mean_squared_error
(mean_squared_error(y_test,reg.predict(X_test)))**0.5

11.102272992160795